In [1]:
#importing libraries
import sklearn
import pandas as pd
import seaborn as sns
import keras as K
import keras.layers as Dense
import keras.models as Sequential
import keras.optimizers as Adam
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [2]:
data_location = 'sqlite:///../data_v2/avocado.db'
data = pd.read_sql('SELECT * FROM avocado', data_location)

In [3]:
data.head(10)

,,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.7,109149.67,130.5,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.0,71976.41,72.58,5811.16,5677.4,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.6,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
5,5,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,2015,Albany
6,6,2015-11-15,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05,0.0,conventional,2015,Albany
7,7,2015-11-08,0.98,109428.33,703.75,101815.36,80.0,6829.22,6266.85,562.37,0.0,conventional,2015,Albany
8,8,2015-11-01,1.02,99811.42,1022.15,87315.57,85.34,11388.36,11104.53,283.83,0.0,conventional,2015,Albany
9,9,2015-10-25,1.07,74338.76,842.4,64757.44,113.0,8625.92,8061.47,564.45,0.0,conventional,2015,Albany


In [4]:
data = data.iloc[: , 1:]

In [5]:
#seperating the prices to be predicted
y = data.AveragePrice
data.drop(['AveragePrice'], axis=1, inplace=True)

In [6]:
data = data.astype({'Date': 'object', 'Total Volume' : 'float64', '4046' : 'float64', '4225' : 'float64', '4770' : 'float64', 'Total Bags' : 'float64', 'Small Bags' : 'float64', 'Large Bags' : 'float64', 'XLarge Bags' : 'float64', 'type' : 'object', 'year' : 'int', 'region':'object' })

In [7]:
#train-test split
#splitting the data into training and test datasets
from sklearn.model_selection import train_test_split

trainflights, testflights, ytrain, ytest = train_test_split(data, y, train_size=0.7,test_size=0.3, random_state=0)

In [8]:
s = (trainflights.dtypes == 'object')
object_cols = list(s[s].index)

n = (trainflights.dtypes == ('float64','int64'))
numerical_cols = list(n[n].index)

In [9]:
#checking the columns containing categorical columns:
print(object_cols)

['Date', 'type', 'region']


In [10]:
#using One Hot Encoder to make the categorical columns usable

oneHot = OneHotEncoder(handle_unknown = 'ignore', sparse=False)
oneHottrain = pd.DataFrame(oneHot.fit_transform(trainflights[object_cols]))
oneHottest = pd.DataFrame(oneHot.transform(testflights[object_cols]))

#reattaching index since OneHotEncoder removes them:
oneHottrain.index = trainflights.index
oneHottest.index = testflights.index

#dropping the old categorical columns:
cattraincol = trainflights.drop(object_cols, axis=1)
cattestcol = testflights.drop(object_cols, axis=1)

#concatenating the new columns:
trainflights = pd.concat([cattraincol, oneHottrain], axis=1)
testflights = pd.concat([cattestcol, oneHottest], axis=1)

In [11]:
#scaling the values

trainf = trainflights.values
testf = testflights.values

minmax = MinMaxScaler()

trainflights = minmax.fit_transform(trainf)
testflights = minmax.transform(testf)

#defining a way to find Mean Absolute Percentage Error:
def PercentError(preds, ytest):
  error = abs(preds - ytest)

  errorp = np.mean(100 - 100*(error/ytest))

  print('the accuracy is:', errorp)

In [12]:
#implementing the algo:
model = RandomForestRegressor(n_estimators=100, random_state=0, verbose=1)

#fitting the data to random forest regressor:
model.fit(trainflights, ytrain)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   13.2s finished


RandomForestRegressor(random_state=0, verbose=1)

In [13]:
#predicting the test dataset:
preds = model.predict(testflights)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished


In [14]:
testflights

array([[6.90448956e-05, 6.28361995e-05, 8.48036854e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.04983851e-03, 2.26192834e-04, 5.83556075e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.67243653e-02, 4.85976831e-03, 1.72295234e-02, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.20848327e-03, 2.04921690e-03, 4.00552743e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.39516087e-04, 8.12261011e-04, 4.59045293e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.15962190e-03, 2.01183793e-04, 1.69275343e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [15]:
ytest = ytest.astype('float')

In [16]:
PercentError(preds, ytest)

the accuracy is: 92.16058342518666


In [27]:
data

,Date,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-04,17074.83,2046.96,1529.20,0.00,13498.67,13066.82,431.85,0.0,organic,2018,WestTexNewMexico
18245,2018-01-28,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,2018-01-21,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,2018-01-14,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico


In [18]:
datac = data
datac = datac.set_index('Date')

In [19]:
data[data.Date == '2015-12-06']

,Date,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
3,2015-12-06,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.00,conventional,2015,Albany
55,2015-12-06,357636.82,283024.01,23740.85,181.92,50690.04,37032.67,13654.66,2.71,conventional,2015,Atlanta
107,2015-12-06,649141.25,51129.29,453586.50,25760.49,118664.97,117112.77,1552.20,0.00,conventional,2015,BaltimoreWashington
159,2015-12-06,95295.34,35590.98,12526.50,4086.26,43091.60,42734.53,0.00,357.07,conventional,2015,Boise
211,2015-12-06,488679.31,5126.32,407520.22,142.99,75889.78,75666.22,223.56,0.00,conventional,2015,Boston
...,...,...,...,...,...,...,...,...,...,...,...,...
11625,2015-12-06,2339.64,733.63,1478.08,0.00,127.93,24.55,103.38,0.00,organic,2015,StLouis
11677,2015-12-06,1803.00,0.00,54.03,0.00,1748.97,1738.06,10.91,0.00,organic,2015,Syracuse
11729,2015-12-06,1739.62,919.89,16.40,0.00,803.33,803.33,0.00,0.00,organic,2015,Tampa
11781,2015-12-06,514112.96,90203.21,212582.74,4066.09,207260.92,108684.49,98576.43,0.00,organic,2015,TotalUS


In [28]:
datac.loc['2015-02-08']

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
Date,,,,,,,,,,,
2015-02-08,51253.97,1357.37,39111.81,163.25,10621.54,10113.10,508.44,0.0,conventional,2015,Albany
2015-02-08,433883.91,377256.38,17162.50,524.85,38940.18,18044.41,20895.77,0.0,conventional,2015,Atlanta
2015-02-08,844931.21,71703.52,554857.41,37437.32,180932.96,175771.39,5161.57,0.0,conventional,2015,BaltimoreWashington
2015-02-08,69970.75,54229.34,9647.24,3047.00,3047.17,1373.42,1673.75,0.0,conventional,2015,Boise
2015-02-08,609985.34,7653.19,495731.70,383.73,106216.72,105403.38,813.34,0.0,conventional,2015,Boston
...,...,...,...,...,...,...,...,...,...,...,...
2015-02-08,1283.80,0.00,114.59,0.00,1169.21,1169.21,0.00,0.0,organic,2015,Syracuse
2015-02-08,2953.53,883.00,7.20,0.00,2063.33,2063.33,0.00,0.0,organic,2015,Tampa
2015-02-08,730874.31,215657.99,273897.84,5316.55,236001.93,179887.47,56114.46,0.0,organic,2015,TotalUS


In [29]:
datacl = data[data['Date'] == '2015-02-08']
datacll = datacl[datacl.region == 'Albany'].where(datacl.type == 'conventional').dropna()
datacll
# datacl.where(datac['region'] == 'Albany')

,Date,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
46,2015-02-08,51253.97,1357.37,39111.81,163.25,10621.54,10113.1,508.44,0.0,conventional,2015.0,Albany


In [46]:
testflights[datacll.index[0]]

array([1.03962044e-03, 1.45057066e-04, 9.60234009e-04, 4.40890181e-05,
       2.17629472e-03, 1.36080107e-03, 4.18735539e-03, 0.00000000e+00,
       6.66666667e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [45]:
print(data.head(5).to_dict())

{'Date': {0: '2015-12-27', 1: '2015-12-20', 2: '2015-12-13', 3: '2015-12-06', 4: '2015-11-29'}, 'Total Volume': {0: 64236.62, 1: 54876.98, 2: 118220.22, 3: 78992.15, 4: 51039.6}, '4046': {0: 1036.74, 1: 674.28, 2: 794.7, 3: 1132.0, 4: 941.48}, '4225': {0: 54454.85, 1: 44638.81, 2: 109149.67, 3: 71976.41, 4: 43838.39}, '4770': {0: 48.16, 1: 58.33, 2: 130.5, 3: 72.58, 4: 75.78}, 'Total Bags': {0: 8696.87, 1: 9505.56, 2: 8145.35, 3: 5811.16, 4: 6183.95}, 'Small Bags': {0: 8603.62, 1: 9408.07, 2: 8042.21, 3: 5677.4, 4: 5986.26}, 'Large Bags': {0: 93.25, 1: 97.49, 2: 103.14, 3: 133.76, 4: 197.69}, 'XLarge Bags': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0}, 'type': {0: 'conventional', 1: 'conventional', 2: 'conventional', 3: 'conventional', 4: 'conventional'}, 'year': {0: 2015, 1: 2015, 2: 2015, 3: 2015, 4: 2015}, 'region': {0: 'Albany', 1: 'Albany', 2: 'Albany', 3: 'Albany', 4: 'Albany'}}


In [ ]:
from datetime import datetime

datetime.strptime('2015-12-06', '%Y-%m-%d')

In [24]:
data.loc[78992.15]

KeyError: 78992.15

In [ ]:
#using linear regression:
LinearModel = LinearRegression()
LinearModel.fit(trainflights, ytrain)

In [ ]:
#predicting on the test dataset:
LinearPredictions = LinearModel.predict(testflights)
PercentError(LinearPredictions, ytest)

I got to this point but I think that train_test_split is not allowed to be used on time series data. I am pretty sure that this is not time series data but it is very similar region wise I guess. Let's just assume that data is not time correlated in any way.

On the other hand how can it not be correlated? As we know what the price will be in August and in December, predicting the price in the October is much easier that way, that is why maybe just going for the train test split scews the correctness of the predictions by a long shot. What if I use TimeSeriesSplit instead?TimeSeriesSplit

In [ ]:
import pickle

# pickle.dump(model, open('../data_v2/model.pkl', 'wb'))